In [1]:
import pandas as pd
import os
import glob
from datetime import datetime
import re

def processar_dados_estoque_novo(caminho_pasta):
    """Lê o conjunto de arquivos de estoque mais recente da pasta e o prepara para a análise."""
    print("\nCarregando dados do novo estoque...")
    todos_arquivos = glob.glob(os.path.join(caminho_pasta, '**/*.csv'), recursive=True)
    if not todos_arquivos: return pd.DataFrame()

    data_recente = None
    regex_data = re.compile(r'(\d{2}\.\d{2}\.\d{2})')

    # Encontra a data mais recente entre todos os arquivos da pasta
    for arquivo in todos_arquivos:
        match = regex_data.search(os.path.basename(arquivo))
        if match:
            data_obj = datetime.strptime(match.group(1), '%d.%m.%y')
            if data_recente is None or data_obj > data_recente: 
                data_recente = data_obj

    if data_recente is None: return pd.DataFrame()

    data_recente_str = data_recente.strftime('%d.%m.%y')
    print(f"Usando o conjunto de estoque da data: {data_recente_str}")

    # Filtra apenas os arquivos que correspondem à data mais recente
    arquivos_para_ler = [f for f in todos_arquivos if data_recente_str in os.path.basename(f)]
    
    lista_dfs_partes = []
    for arquivo in arquivos_para_ler:
        try:
            # Lê cada arquivo CSV, importando todas as suas colunas
            df_parte = pd.read_csv(arquivo, sep=';', decimal=',', encoding='utf-8', on_bad_lines='warn')
            lista_dfs_partes.append(df_parte)
        except Exception as e:
            print(f"Aviso: Não foi possível ler o arquivo {arquivo}. Erro: {e}")
            continue

    if not lista_dfs_partes: return pd.DataFrame()

    # Concatena todos os dataframes lidos em um único
    df_consolidado = pd.concat(lista_dfs_partes, ignore_index=True)
    
    # Renomeia colunas específicas para padronização
    df_consolidado.rename(columns={
        'DataVencimento': 'Data Vencimento', 
        'ValorPresente': 'Valor Presente',
        'ValorNominal': 'Valor Nominal'
    }, inplace=True)
    
    # Converte as colunas de data e valores numéricos
    df_consolidado['Data Vencimento'] = pd.to_datetime(df_consolidado['Data Vencimento'], dayfirst=True, errors='coerce')
    df_consolidado['Valor Presente'] = pd.to_numeric(df_consolidado['Valor Presente'], errors='coerce')
    df_consolidado['Valor Nominal'] = pd.to_numeric(df_consolidado['Valor Nominal'], errors='coerce')
    
    print(f"Dados do novo estoque consolidados: {len(df_consolidado)} linhas.")
    return df_consolidado

# --- Exemplo de como usar a função ---

# 1. Defina o caminho para a pasta onde estão seus arquivos de estoque
PATH_ESTOQUE_NOVO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Estoques"

# 2. Chame a função para carregar os dados
df_estoque_carregado = processar_dados_estoque_novo(PATH_ESTOQUE_NOVO)

# 3. Exiba as primeiras linhas do DataFrame resultante
print("\nResultado do carregamento:")
print(df_estoque_carregado.head())


Carregando dados do novo estoque...
Usando o conjunto de estoque da data: 30.09.25
Dados do novo estoque consolidados: 1724011 linhas.

Resultado do carregamento:
       Situacao PES_TIPO_PESSOA      CedenteCnpjCpf  TIT_CEDENTE_ENT_CODIGO  \
0  Sem cobrança               J  34.337.707/0001-00                  318853   
1  Sem cobrança               J  34.337.707/0001-00                  318853   
2  Sem cobrança               J  34.337.707/0001-00                  318853   
3  Sem cobrança               J  34.337.707/0001-00                  318853   
4  Sem cobrança               J  34.337.707/0001-00                  318853   

                                      CedenteNome     Cnae  \
0  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6499999   
1  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6499999   
2  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6499999   
3  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6499999   
4  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  

In [2]:
df_estoque_carregado.columns

Index(['Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf',
       'TIT_CEDENTE_ENT_CODIGO', 'CedenteNome', 'Cnae', 'SecaoCNAEDescricao',
       'NotaPdd', 'SAC_TIPO_PESSOA', 'SacadoCnpjCpf', 'SacadoNome',
       'IdTituloVortx', 'TipoAtivo', 'DataEmissao', 'DataAquisicao',
       'Data Vencimento', 'NumeroBoleto', 'NumeroTitulo', 'CampoChave',
       'ValorAquisicao', 'Valor Nominal', 'Valor Presente', 'PDDNota',
       'PDDVencido', 'PagamentoParcial', 'Coobricacao', 'DataGeracao',
       'PDDTotal', 'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3',
       'CampoAdicional4', 'CampoAdicional5', 'PDDEfeitoVagao',
       'PercentagemEfeitoVagao', 'IdTituloVortxOriginador', 'Registradora',
       'IdContratoRegistradora', 'IdTituloRegistradora'],
      dtype='object')